In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
from dateutil.relativedelta import *
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from collections import defaultdict

### Import Data

In [2]:
# Get historical crypto market cap rank data
cryptoMarketCapRankDf = pd.read_csv('data\CryptoMarketCap.csv')
cryptoMarketCapRankDf['Date'] = pd.to_datetime(cryptoMarketCapRankDf['Date'])

In [3]:
# Get historical crypto price data
cryptoPriceDf = pd.read_csv('data\TradingViewCryptoPrice.csv', index_col=0)
cryptoPriceDf.index = pd.to_datetime(cryptoPriceDf.index)

### Formation Period

In [15]:
# Formation period paramenter
cutoffDate = '2021-01-01'
cutoffRank = 50

In [16]:
# Get formation period sample crypto list
marketCapCutoffDate = pd.to_datetime(cutoffDate) - timedelta(days=1)
sampleCrypto = cryptoMarketCapRankDf[(cryptoMarketCapRankDf['Date'] == marketCapCutoffDate) & (cryptoMarketCapRankDf['Rank'] <= cutoffRank)]
sampleCrypto = list(sampleCrypto['Symbol'])

In [17]:
# Fliter the crypto with formation period and availiablity of crpyto price
sampleCryptoPrice = cryptoPriceDf.loc[pd.to_datetime(cutoffDate) + relativedelta(months=-12): marketCapCutoffDate]
sampleCrypto = set(sampleCrypto).intersection([x[7:-3] for x in sampleCryptoPrice.columns])

# Data Cleaning
sampleCryptoPrice = sampleCryptoPrice[["CRYPTO:" + x + "USD" for x in sampleCrypto]]
sampleCryptoPrice = sampleCryptoPrice.ffill(axis=0)
sampleCryptoPrice = sampleCryptoPrice.dropna(axis=1)

print('Remaining number of crpyto: ', len(sampleCryptoPrice.columns))

Remaining number of crpyto:  34


#### Cointegration Method

In [18]:
ADFtestThreshold = 0.01

In [19]:
# Take log for the price series
sampleCryptoLogPrice = np.log(sampleCryptoPrice)

In [20]:
# Test all price series for order 1 integration
I0Series = []
for crypto in sampleCryptoLogPrice.columns:
    if adfuller(sampleCryptoLogPrice[crypto])[1] < ADFtestThreshold:
        I0Series.append(crypto)

# remove price series with order 0 integration from samples
sampleCryptoPrice = sampleCryptoPrice.drop(I0Series, axis=1)

print('I0Series: ', I0Series)
print('Remaining number of crpyto: ', len(sampleCryptoPrice.columns))

I0Series:  ['CRYPTO:BSVUSD', 'CRYPTO:DASHUSD']
Remaining number of crpyto:  32


In [21]:
# Finding cointegrated pairs
CointegratedPairs = pd.DataFrame(columns=['Crypto 1', 'Crypto 2', 'Constant', 'Beta', 'Spread mean', 'Spread std'])
for crypto1 in sampleCryptoPrice.columns:
    for crypto2 in sampleCryptoPrice.columns:
        if crypto1 != crypto2:
            # OLS regression input
            y = sampleCryptoPrice[crypto1]
            x = sampleCryptoPrice[crypto2]
            x_withConst = sm.add_constant(x)

            # OLS Regression fitting
            model = sm.OLS(y, x_withConst).fit()

            # OLS Result
            const = model.params[0]
            beta = model.params[1]
            residuals = y - (x * beta + const)

            # the residuals are tested for stationarity by using the Augmented-Dickey-Fuller test (ADF-test)
            if adfuller(residuals)[1] < ADFtestThreshold:
                spread = sampleCryptoPrice[crypto1] - beta * sampleCryptoPrice[crypto2]
                CointegratedPairs.loc[len(CointegratedPairs)] = {'Crypto 1': crypto1, 'Crypto 2': crypto2, 'Constant': const, 'Beta': beta, 'Spread mean': spread.mean(), 'Spread std': spread.std()}

In [22]:
CointegratedPairs

,Crypto 1,Crypto 2,Constant,Beta,Spread mean,Spread std
0,CRYPTO:ETCUSD,CRYPTO:AAVEUSD,6.990717,-0.013798,6.990717,1.554745
1,CRYPTO:ETCUSD,CRYPTO:SNXUSD,7.096420,-0.186560,7.096420,1.563580
2,CRYPTO:ETCUSD,CRYPTO:ADAUSD,6.876213,-3.312202,6.876213,1.599083
3,CRYPTO:ETCUSD,CRYPTO:XMRUSD,6.843248,-0.002998,6.843248,1.602418
4,CRYPTO:ETCUSD,CRYPTO:ZILUSD,6.940620,-21.307243,6.940620,1.580385
...,...,...,...,...,...,...
57,CRYPTO:XLMUSD,CRYPTO:ZILUSD,0.049693,1.977547,0.049693,0.023341
58,CRYPTO:XLMUSD,CRYPTO:THETAUSD,0.053958,0.078079,0.053958,0.026118
59,CRYPTO:XLMUSD,CRYPTO:BTCUSD,0.006997,0.000007,0.006997,0.019124
60,CRYPTO:THETAUSD,CRYPTO:KSMUSD,0.104725,0.014875,0.104725,0.131586


### Trading Period (Need to consider close and open new positions in the same day)

In [29]:
spreadThreshold = 2.5
closeThreshold = 0

In [30]:
TransactionRecords = pd.DataFrame(columns=['Date', 'Crypto', 'Long/Short', 'Price', "Open/Close", "Transaction pair", "Round Trip No.", "Pair No.", "Hedge Ratio"])
SpreadRecords = pd.DataFrame()
PairNo = 0

# get trading crpyto price
tradingCryptoPrice = cryptoPriceDf.loc[pd.to_datetime(cutoffDate): pd.to_datetime(cutoffDate) + relativedelta(days=59)]
# logTradingCryptoPrice = np.log(tradingCryptoPrice)


for i in range(len(CointegratedPairs)):
    # parameter
    crypto1 = CointegratedPairs.loc[i, 'Crypto 1']
    crypto2 = CointegratedPairs.loc[i, 'Crypto 2']
    # const = CointegratedPairs.loc[i, 'Constant']
    beta = CointegratedPairs.loc[i, 'Beta']
    spreadMean = CointegratedPairs.loc[i, 'Spread mean']
    spreadStd = CointegratedPairs.loc[i, 'Spread std']

    # calculate spread
    spread = tradingCryptoPrice[crypto1] - tradingCryptoPrice[crypto2] * beta
    normalizedSpread = (spread - spreadMean) / spreadStd

    # check if there is any trading opportunity
    if len(normalizedSpread[(normalizedSpread >= spreadThreshold) | (normalizedSpread <= -spreadThreshold)]) > 0:
        
        # save the spread records
        normalizedSpread.name = crypto1 + " " + crypto2
        SpreadRecords = SpreadRecords.merge(normalizedSpread, how='outer', left_index=True, right_index=True)

        # initialize before the transaction
        PairNo += 1
        normalizedSpread.name = 'spread'
        normalizedSpread = normalizedSpread.to_frame()
        normalizedSpread['position'] = 0
        RoundTripNo = 1

        for date in normalizedSpread.index:
            # When the trading date is not the last day
            if date != (pd.to_datetime(cutoffDate) + relativedelta(days=59)):
                
                # continuous the position if the spread do not cross closeThreshold
                if normalizedSpread.loc[date, 'position'] == -1 and normalizedSpread.loc[date, 'spread'] > closeThreshold:
                    normalizedSpread.loc[date + relativedelta(days=1), 'position'] = -1
                
                # continuous the position if the spread do not cross closeThreshold
                elif normalizedSpread.loc[date, 'position'] == 1 and normalizedSpread.loc[date, 'spread'] < closeThreshold:
                    normalizedSpread.loc[date + relativedelta(days=1), 'position'] = 1
                
                # short crypto 1 and long crypto 2 if spread >= spreadThreshold
                elif normalizedSpread.loc[date, 'spread'] >= spreadThreshold:
                    normalizedSpread.loc[date + relativedelta(days=1), 'position'] = -1
                    # Long/Short with tomorrow open price  i.e. today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date + relativedelta(days=1), crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Open", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date + relativedelta(days=1), crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Open", crypto1, RoundTripNo, PairNo, beta]
                
                # long crypto 1 and short crypto 2 if spread <= -spreadThreshold
                elif normalizedSpread.loc[date, 'spread'] <= -spreadThreshold:
                    normalizedSpread.loc[date + relativedelta(days=1), 'position'] = 1
                    # Long/Short with tomorrow open price  i.e. today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date + relativedelta(days=1), crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Open", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date + relativedelta(days=1), crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Open", crypto1, RoundTripNo, PairNo, beta]
                
                # Close the position if the spread cross closeThreshold
                elif normalizedSpread.loc[date, 'position'] == -1 and normalizedSpread.loc[date, 'spread'] <= closeThreshold:
                    # Long/Short with today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo, beta]
                    RoundTripNo += 1

                elif normalizedSpread.loc[date, 'position'] == 1 and normalizedSpread.loc[date, 'spread'] >= closeThreshold:
                    # Long/Short with today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo, beta]
                    RoundTripNo += 1
            
            # For last day closing position
            else:
                if normalizedSpread.loc[date, 'position'] == -1:
                    # Long/Short with today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Long",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Short",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo, beta]
                    RoundTripNo += 1
                elif normalizedSpread.loc[date, 'position'] == 1:
                    # Long/Short with today close price
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto1, "Short",  tradingCryptoPrice.loc[date, crypto1], "Close", crypto2, RoundTripNo, PairNo, 1]
                    TransactionRecords.loc[len(TransactionRecords)] = [date, crypto2, "Long",  tradingCryptoPrice.loc[date, crypto2], "Close", crypto1, RoundTripNo, PairNo, beta]
                    RoundTripNo += 1

In [31]:
result = pd.DataFrame(columns=['Pair No.', 'Round Trip No.', 'Start Date', 'End Date', 'crypto 1', 'crypto 2', 'crypto 1 return', 'crypto 2 return', 'beta'])
# loop each pair of transactions
for i in range(1, TransactionRecords['Pair No.'].max() + 1):
    pair = TransactionRecords[TransactionRecords['Pair No.'] == i]

    # loop each Round Trip in pair
    for j in range(1, pair['Round Trip No.'].max() + 1):
        roundTrip = pair[pair['Round Trip No.'] == j]

        returnResult = dict()

        # loop each crypto in the round trip
        for crypto in set(roundTrip['Crypto']):
            # prepare the specific round trip transaction record
            record = roundTrip[roundTrip['Crypto'] == crypto]
            record = record.reset_index(drop=True)

            # Calculate the return of the specific round trip
            returns = record['Price'][1] / record['Price'][0] - 1
            if record['Long/Short'][0] == 'Short':
                returns = -returns 

            # Insert Record
            if len(returnResult) == 0:
                returnResult['Pair No.'] = i
                returnResult['Round Trip No.'] = j
                returnResult['Start Date'] = record['Date'][0]
                returnResult['End Date'] = record['Date'][1]
            
            if record['Hedge Ratio'][0] == 1:
                returnResult['crypto 1'] = crypto
                returnResult['crypto 1 return'] = returns
            else:
                returnResult['crypto 2'] = crypto
                returnResult['crypto 2 return'] = returns
                returnResult['beta'] = record['Hedge Ratio'][0]

        result.loc[len(result)] = returnResult  


In [32]:
# Remark: return can be more than -100% for the short selling position
result['Total Return'] = result['crypto 1 return'] + result['crypto 2 return'] * result['beta']
result

,Pair No.,Round Trip No.,Start Date,End Date,crypto 1,crypto 2,crypto 1 return,crypto 2 return,beta,Total Return
0,1,1,2021-01-11,2021-03-01,CRYPTO:ETCUSD,CRYPTO:AAVEUSD,0.036751,2.040670,-0.013798,0.008593
1,2,1,2021-01-11,2021-03-01,CRYPTO:ETCUSD,CRYPTO:ADAUSD,0.036751,3.300806,-3.312202,-10.896186
2,3,1,2021-01-11,2021-03-01,CRYPTO:ETCUSD,CRYPTO:XMRUSD,0.036751,0.243163,-0.002998,0.036022
3,4,1,2021-01-11,2021-03-01,CRYPTO:ETCUSD,CRYPTO:ZILUSD,0.036751,0.578030,-21.307243,-12.279471
4,5,1,2021-01-11,2021-03-01,CRYPTO:ETCUSD,CRYPTO:LINKUSD,0.036751,0.710069,-0.069604,-0.012673
...,...,...,...,...,...,...,...,...,...,...
70,52,1,2021-01-02,2021-01-07,CRYPTO:XLMUSD,CRYPTO:BTCUSD,1.598010,-0.254020,0.000007,1.598009
71,53,1,2021-01-02,2021-02-17,CRYPTO:THETAUSD,CRYPTO:KSMUSD,-0.780024,2.457615,0.014875,-0.743468
72,53,2,2021-02-23,2021-03-01,CRYPTO:THETAUSD,CRYPTO:KSMUSD,-0.014191,0.158501,0.014875,-0.011834
73,54,1,2021-01-11,2021-02-09,CRYPTO:EOSUSD,CRYPTO:ETCUSD,0.362144,0.171475,0.344127,0.421153


In [33]:
result['Total Return'].mean()

-6.299025959895553

In [28]:
SpreadRecords

,CRYPTO:ETCUSD CRYPTO:AAVEUSD,CRYPTO:ETCUSD CRYPTO:ADAUSD,CRYPTO:ETCUSD CRYPTO:XMRUSD,CRYPTO:ETCUSD CRYPTO:ZILUSD,CRYPTO:ETCUSD CRYPTO:LINKUSD,CRYPTO:ETCUSD CRYPTO:BNBUSD,CRYPTO:ETCUSD CRYPTO:FILUSD,CRYPTO:ETCUSD CRYPTO:LEOUSD,CRYPTO:ETCUSD CRYPTO:OKBUSD,CRYPTO:ETCUSD CRYPTO:ETHUSD,...,CRYPTO:WAVESUSD CRYPTO:THETAUSD,CRYPTO:WAVESUSD CRYPTO:BTCUSD,CRYPTO:MKRUSD CRYPTO:ZECUSD,CRYPTO:MKRUSD CRYPTO:MIOTAUSD,CRYPTO:MKRUSD CRYPTO:ATOMUSD,CRYPTO:XLMUSD CRYPTO:ZILUSD,CRYPTO:XLMUSD CRYPTO:THETAUSD,CRYPTO:XLMUSD CRYPTO:BTCUSD,CRYPTO:THETAUSD CRYPTO:KSMUSD,CRYPTO:EOSUSD CRYPTO:ETCUSD
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-01,-0.009637,-0.359428,-0.447076,0.290088,-0.355323,-0.566425,-0.335451,0.044171,-0.477508,-0.176672,...,-4.094915,-4.984335,1.368307,0.523332,-0.356565,-3.135630,-2.896213,-3.869471,6.328334,0.118570
2021-01-02,-0.008135,-0.309065,-0.386602,0.251981,-0.292633,-0.521009,-0.303787,0.096234,-0.444619,-0.091770,...,-6.325724,-6.893735,1.727612,1.121691,0.496323,-2.800670,-4.380147,-5.111658,9.693241,-0.028635
2021-01-03,0.572949,0.246227,0.097710,0.689719,0.275598,-0.032794,0.205380,0.624967,0.036329,0.582779,...,-5.136573,-7.352314,2.598695,1.693084,1.104835,-2.024793,-3.215297,-4.955494,7.203979,-0.450877
2021-01-04,1.052207,0.578254,0.380725,1.016150,0.568229,0.258501,0.508540,0.794235,0.321857,0.931075,...,-4.582367,-7.023871,2.650039,1.026684,1.038264,-1.041812,-1.683414,-3.139854,6.392538,-0.999542
2021-01-05,1.255664,0.796410,0.533416,1.166203,0.760866,0.404914,0.670898,0.926674,0.474115,1.132744,...,-5.142894,-7.815270,3.921597,2.184940,2.065164,0.225615,-1.044331,-2.309928,7.660942,-1.020538
2021-01-06,1.435502,1.162384,0.756366,1.488874,1.096859,0.615506,0.902783,1.293569,0.683890,1.440416,...,-5.085565,-8.927364,7.474350,5.918642,5.427915,5.978623,4.595774,4.541390,7.182864,0.437927
2021-01-07,1.201560,0.866188,0.533758,1.225623,0.811964,0.390465,0.669295,1.017896,0.461089,1.228113,...,-4.235340,-10.170773,8.077097,6.928880,6.780492,4.877824,3.948461,1.984535,6.448823,0.214379
2021-01-08,0.984134,0.702384,0.359353,1.024703,0.603053,0.219122,0.475550,0.865845,0.283884,1.046165,...,-4.486867,-11.132466,7.282382,6.488621,6.389660,4.386260,3.371538,0.782430,6.459694,0.263320
2021-01-09,1.603700,1.299288,0.908298,1.604283,1.255862,0.754399,1.056580,1.447902,0.822488,1.642557,...,-4.255664,-10.227882,14.840767,14.476464,14.034748,4.794458,3.756556,1.681556,6.923984,1.122977
